## Report Milestone 2 Exercise 2 Information Retrieval 2026
This report documents how our group (group 40) completed the second milestone of exercise 2. In general we were tasked to improve the approach used in milestone 1. Where we had to experiment with different configurations for Audio Fingerprinting and Identifiction. The improvments are based on the Fingerprinting-algorithm by Wang (2003).
The goal of milestone 2.2 was to develop a robust pipeline that accurately identifies music tracks even under difficult conditions (noise, cell phone recordings). A particular focus was placed on the efficient storage of fingerprints in an inverted index and scaling the system to a dataset of over 3,000 tracks.

### Task 1: Hash Generation and Storage (30%)

The optimal parameters from milestone 2.1 (k=18, t=15) were used to extract the distinctive points (peaks) from the spectrogram. These parameters define the size of the local neighborhood filter and ensure that only the most significant frequency peaks are selected as anchor points. To increase robustness against temporal and frequency-related distortions, we tested various configurations for the target zone. 

| Configuration | min time | max time | max frequency | Fan-out | Description |
| :--- | :---: | :---: | :---: | :---: | :--- |
| `std` | 10 | 50 | 50 | 3 | basic config|
| `wide_t` | 10 | **100** | 50 | 3 | focus on bigger time window |
| `wide_f` | 10 | 50 | **100** | 3 | focus on bigger frequency window |
| `dense` | 10 | 50 | 50 | **6** | more pairs through increased fanout |
| `super_dense` | 10 | **60** | **100** | **15** | even more increased fanout |

To test all those configuration we implemented a improved script that supports capturing the tracks as hashes.

#### import dependencies

In [2]:
import os
import librosa
import numpy as np
import scipy.ndimage as ndimage
import sys
import pickle
from utils import get_peaks, generate_hashes_from_peaks

#### define configs

In [ ]:
configs = [
    ("std", 10, 50, 50, 3),      # basic config
    ("wide_t", 10, 100, 50, 3),   # focus on bigger time window
    ("wide_f", 10, 50, 100, 3),   # focus on bigger frequency window
    ("dense", 10, 50, 50, 6),      # more pairs through increased fanout
    ("super_dense", 10, 60, 100, 15) # even more increased fanout
]

#### get_peak and generate_hashes_from_peaks functions

### 32-Bit Hash Konstruktion

A key aspect of Wang's algorithm is the representation of pairs (anchor and target points) as compact 32-bit unsigned integers.  
 We have opted for the following bit allocation to achieve an optimal balance between precision and value range:  
 * 10 bits for $f_1$ (anchor frequency): Allows 1024 frequency bins, which covers the full resolution of our STFT spectrogram.   
 * 10 bits for $f_2$ (target frequency): Also 1024 bins for the target point.  
 * 12 bits for $\Delta t$ (time difference): Allows a maximum difference of 4096 frames. With a hop size of 1024, this corresponds to approximately 190 seconds – far more than our maximum target zone width of 60 frames, giving us buffer for future expansions.  
 
 Hash formula: `hash_32 = (f1 | (f2 << 10) | (dt << 20)) & 0xFFFFFFFF`

In [4]:
def get_peaks(Y, k, t, thresh=0.01):
    size = (2*k + 1, 2*t + 1)
    result = ndimage.maximum_filter(Y, size=size, mode='constant')
    cmap = (Y == result) & (Y > thresh)
    return np.argwhere(cmap)

def generate_hashes_from_peaks(peaks, dt_min, dt_max, df_max, fan_out):
    # Sort peaks by time (column 1)
    peaks = peaks[peaks[:, 1].argsort()]

    pairs = []

    for i in range(len(peaks)):
        anchor = peaks[i]
        f1, t1 = anchor[0], anchor[1]
        
        # Look for targets in a window after the anchor
        count = 0
        for j in range(i + 1, len(peaks)):
            target = peaks[j]
            f2, t2 = target[0], target[1]
            dt = t2 - t1
            
            # Check if target is within the Target Zone
            if dt_min <= dt <= dt_max and abs(f2 - f1) <= df_max:
                # 1. Create the 32-bit Hash
                # Example: pack f1 (10 bits), f2 (10 bits), and dt (12 bits)

                hash_32 = (f1 | (f2 << 10) | (dt << 20)) & 0xFFFFFFFF
                
                # 2. Store (hash, t1)
                pairs.append((hash_32, t1))
                
                count += 1
                if count >= fan_out:
                    break
    return pairs

#### iterrate over all songs and process them and save in inverted index as pickle file

We utilized Python dictionaries, where the keys are the generated 32-bit hashes and the values are he Key is the unique 32-bit hash nd the Value is a list containing multiple tuples. Each tuple represents an individual 'hit' or occurrence, allowing the system to track every song and every timestamp where that specific hash appears. Each uple contains (song_id, offset_time).

 To maximize efficiency, a separate Song Map stores to store the specic song names, ensuring the main index remains lean by only holding integer-based pointers. Unlike linear arrays, which require slow step-by-step searching, or SQL databases that carry high processing overhead, the dictionary approach handles the millions of entries in Task 3 with minimal latency. This architecture ensures that even with 3,319 tracks, the system can jump directly to relevant candidates during the retrieval phase.

 To preserve the inverted index for large-scale use, we serialized the Python dictionary into Pickle files, which allow for the direct saving and loading of complex data structures without manual conversion. Unlike plain text or CSV formats, Pickle maintains the specific hierarchy of our "key-to-list" mapping, ensuring the dictionary is ready for immediate use upon loading. We specifically prioritized Standard Pickle (Protocol 4/5) over compressed formats like bz2 to maximize I/O speed, significantly reducing the time required to save millions of hashes during the Task 3 scale-up. This approach was essential for the 3,319-track dataset, as it allowed the desktop hardware to handle large write operations without the CPU bottlenecks associated with heavy compression.

In [6]:
def run_indexing(config_name, dt_min, dt_max, df_max, fan_out):
    audio_folder = 'data/40'
    song_files = [f for f in os.listdir(audio_folder) if f.endswith('.mp3')]
    inverted_index = {}
    song_map = {}

    for song_id, song in enumerate(song_files):
        song_map[song_id] = song
        path = os.path.join(audio_folder, song)
        
        # load songs (full length)
        y, sr = librosa.load(path, sr=22050)
        Y = np.abs(librosa.stft(y, n_fft=2048, hop_length=1024))
        
        peaks = get_peaks(Y, 18, 15)
        pairs = generate_hashes_from_peaks(peaks, dt_min, dt_max, df_max, fan_out)

        for h, t1 in pairs:
            if h not in inverted_index:
                inverted_index[h] = []
            inverted_index[h].append((song_id, t1))

    # calculate some statistics
    total_unique = len(inverted_index)
    total_entries = sum(len(v) for v in inverted_index.values())
    avg_per_track = total_entries / len(song_files) if song_files else 0
    size_mb = sys.getsizeof(inverted_index) / (1024**2)

    # save as pickle file
    filename = f"db_{config_name}.pkl"
    with open(filename, 'wb') as f:
        pickle.dump({'index': inverted_index, 'map': song_map}, f)
    
    print(f" > Config {config_name} finished and saved.")

    # return stats
    return {
        "name": config_name,
        "unique": total_unique,
        "total": total_entries,
        "avg": avg_per_track,
        "size": size_mb
    }

def task1():
    all_stats = []
    
    print("Starting indexing for all configurations...")
    for name, dt_min, dt_max, df_max, fan_out in configs:
        stats = run_indexing(name, dt_min, dt_max, df_max, fan_out)
        all_stats.append(stats)

    # final summary for stats
    print("\n" + "="*80)
    print(f"{'CONFIG':<15} | {'UNIQUE HASHES':<15} | {'TOTAL ENTRIES':<15} | {'AVG/TRACK':<12} | {'SIZE (MB)':<10}")
    print("-" * 80)
    for s in all_stats:
        print(f"{s['name']:<15} | {s['unique']:<15,} | {s['total']:<15,} | {s['avg']:<12.1f} | {s['size']:<10.2f}")
    print("="*80)

task1()

Starting indexing for all configurations...
 > Config std finished and saved.
 > Config wide_t finished and saved.
 > Config wide_f finished and saved.
 > Config dense finished and saved.
 > Config super_dense finished and saved.

CONFIG          | UNIQUE HASHES   | TOTAL ENTRIES   | AVG/TRACK    | SIZE (MB) 
--------------------------------------------------------------------------------
std             | 2,184,674       | 5,341,826       | 9306.3       | 80.00     
wide_t          | 2,897,206       | 6,369,372       | 11096.5      | 160.00    
wide_f          | 3,007,011       | 6,236,003       | 10864.1      | 160.00    
dense           | 2,442,910       | 6,614,762       | 11524.0      | 80.00     
super_dense     | 6,159,719       | 16,258,554      | 28325.0      | 320.00    


The evaluation of five distinct configurations revealed a clear trade-off between fingerprint density and resource consumption. While the std configuration was the most lightweight with ~9,306 entries per track, the super_dense configuration increased this density by over 300%, resulting in a massive 16.2 million total entries. The avg/track category informs how many hash entries where saved in the dictionary on average.

 * Density vs. Size: As density increased, the storage footprint grew proportionally, peaking at 320MB for the super_dense model.

 * Search Robustness: The wide_t and wide_f configurations increased the "Target Zone," which produced more unique hashes and improved the system's ability to handle time-stretching or frequency shifts.

 * Optimal Balance: The dense configuration provided a significant boost in entries (11,524 per track) while maintaining the same 80MB storage footprint as the standard model, demonstrating high indexing efficiency.

 * Scalability Insight: These results directly informed the choice of the smart_dense configuration for Task 3, ensuring enough redundancy for a 75% mobile-query accuracy (see task 2) without exceeding storage capabilities of our desktop hardware.

## Task 2: Audio Identification (30%)

In Milestone 2.1, audio identification was performed by directly matching
constellation maps.
In this task, we instead use **fingerprint hashes** generated from peak pairs.

Each hash represents:
- Two frequency bins
- Their time difference

Hashes are stored in an inverted index and matched using
**temporal offset consistency voting**.


In [44]:
import re
import time
import glob
import csv
from dataclasses import dataclass, asdict
from typing import Dict, List, Tuple, Optional
from collections import defaultdict, Counter

## Paths and Database Configuration

The fingerprint databases were generated in Task 1 and stored as pickle files.
Each database corresponds to a different fingerprint configuration.


In [45]:
QUERY_ROOT = "queries"
OUT_DIR = "ms22_task2_results"
os.makedirs(OUT_DIR, exist_ok=True)

DB_PKLS = {
    "std": "db_std.pkl",
    "wide_t": "db_wide_t.pkl",
    "wide_f": "db_wide_f.pkl",
    "dense": "db_dense.pkl",
    "super_dense": "db_super_dense.pkl"
}


## Audio and Peak Detection Parameters

All parameters must match those used during fingerprint indexing (Task 1).
Peak detection parameters are taken from the best-performing configuration
identified in Milestone 2.1.


In [46]:
SR = 22050
N_FFT = 2048
HOP = 1024
BIN_MAX = 128

Q_DURATION = 10.0

PEAK_K = 18
PEAK_T = 15
PEAK_THRESH = 0.01


## Query Sets

Queries are grouped by distortion type:
- Original
- Noise
- Coding
- Mobile

These match the setup used in Milestone 2.1.


In [47]:
QUERY_SUBFOLDERS = {
    "Original": "Original",
    "Noise": "Noise",
    "Coding": "Coding",
    "Mobile": "Mobile",
}

def collect_queries(query_root: str) -> Dict[str, List[str]]:
    out = {}
    for dtype, sub in QUERY_SUBFOLDERS.items():
        folder = os.path.join(query_root, sub)
        ext = "*.wav" if dtype != "Coding" else "*.mp3"
        out[dtype] = sorted(glob.glob(os.path.join(folder, ext)))
    return out


## Ground Truth Extraction

The ground truth track ID is extracted from the query filename.
A query is counted as correct if the predicted filename matches the ground truth.


In [48]:
def extract_track_id(path: str) -> Optional[str]:
    base = os.path.basename(path)
    m = re.search(r"(\d+)", base)
    return m.group(1) if m else None

def gt_db_filename_from_id(track_id: str) -> str:
    return f"{track_id}.mp3"


## Hash Database Structure

Each database pickle contains:
- `index`: dictionary mapping hash → list of (song_id, time_offset)
- `map`: dictionary mapping song_id → filename

This corresponds to an inverted index as described by Müller.


In [49]:
def load_hash_db(path: str):
    with open(path, "rb") as f:
        obj = pickle.load(f)
    return obj["index"], obj["map"]


## Hash-Based Matching Strategy

For each query:
1. Compute STFT magnitude
2. Detect spectral peaks
3. Generate fingerprint hashes
4. Look up hashes in the database
5. Vote for (song_id, Δt) pairs
6. Select the song with the highest consistent offset vote

This is the classical **Wang offset-voting scheme**.


In [50]:
def identify_query_hash(
    query_path,
    index,
    song_map,
    dt_min,
    dt_max,
    df_max,
    fan_out
):
    y, _ = librosa.load(query_path, sr=SR, mono=True, duration=Q_DURATION)
    Y = np.abs(librosa.stft(y, n_fft=N_FFT, hop_length=HOP))

    peaks = get_peaks(Y, PEAK_K, PEAK_T, PEAK_THRESH)
    pairs = generate_hashes_from_peaks(peaks, dt_min, dt_max, df_max, fan_out)

    votes = Counter()

    for h, t_q in pairs:
        if h not in index:
            continue
        for song_id, t_db in index[h]:
            votes[(song_id, t_db - t_q)] += 1

    if not votes:
        return "", 0

    best_song = max(
        votes,
        key=lambda x: votes[x]
    )[0]

    return song_map[best_song], max(votes.values())


## Evaluation Metrics

For each configuration and distortion type we measure:
- Number of correct identifications (hits)
- Accuracy
- Average query time
- Average maximum offset vote

Results are stored for later analysis and reporting.


In [53]:
@dataclass
class Row:
    distortion: str
    config: str
    dt_min: int
    dt_max: int
    df_max: int
    fan_out: int
    hits: int
    total: int
    accuracy: float
    avg_query_time_s: float
    avg_best_vote: float

def write_csv(path: str, rows: List[Row]):
    if not rows:
        return
    with open(path, "w", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=list(asdict(rows[0]).keys()))
        w.writeheader()
        for r in rows:
            w.writerow(asdict(r))

In [54]:
def task2():
    queries = collect_queries(QUERY_ROOT)
    for dtype, qfiles in queries.items():
        print(f"{dtype}: {len(qfiles)} queries found", flush=True)

    # Same 4 configs as your colleague used for indexing:
    # (name, dt_min, dt_max, df_max, fan_out)
    CONFIGS = [
        ("std",    10,  50,  50, 3),
        ("wide_t", 10, 100,  50, 3),
        ("wide_f", 10,  50, 100, 3),
        ("dense",  10,  50,  50, 6),
        ("super_dense", 10, 60, 100, 15)
    ]

    all_rows: List[Row] = []

    # also create a combined pool of queries
    combined_queries: List[Tuple[str, str]] = []
    for dist, files in queries.items():
        for p in files:
            combined_queries.append((dist, p))

    for cfg_name, dt_min, dt_max, df_max, fan_out in CONFIGS:
        db_path = DB_PKLS[cfg_name]
        print(f"\n=== HASH CONFIG {cfg_name} ({db_path}) ===", flush=True)

        index, song_map = load_hash_db(db_path)
        print(f"Loaded index: {len(index)} unique hashes, {len(song_map)} songs", flush=True)
        # per distortion
        for distortion, qfiles in queries.items():
            if not qfiles:
                continue

            hits = 0
            times = []
            votes = []
            total = len(qfiles)

            for qpath in qfiles:
                tid = extract_track_id(qpath)
                if tid is None:
                    continue
                gt = gt_db_filename_from_id(tid)

                t0 = time.time()
                pred, best_vote = identify_query_hash(
                    query_path=qpath,
                    index=index,
                    song_map=song_map,
                    dt_min=dt_min,
                    dt_max=dt_max,
                    df_max=df_max,
                    fan_out=fan_out,
                )
                dt = time.time() - t0
                times.append(dt)
                votes.append(best_vote)

                if pred == gt:
                    hits += 1

            avg_t = float(np.mean(times)) if times else 0.0
            avg_vote = float(np.mean(votes)) if votes else 0.0

            row = Row(
                distortion=distortion,
                config=cfg_name,
                dt_min=dt_min,
                dt_max=dt_max,
                df_max=df_max,
                fan_out=fan_out,
                hits=hits,
                total=total,
                accuracy=hits / total if total else 0.0,
                avg_query_time_s=avg_t,
                avg_best_vote=avg_vote,
            )
            all_rows.append(row)
            print(f"  => {distortion}: hits {hits}/{total}, acc={row.accuracy:.2f}, avg_time={avg_t:.2f}s, avg_vote={avg_vote:.1f}", flush=True)

        # combined
        hits = 0
        times = []
        votes = []
        total = len(combined_queries)

        for distortion, qpath in combined_queries:
            tid = extract_track_id(qpath)
            if tid is None:
                continue
            gt = gt_db_filename_from_id(tid)

            t0 = time.time()
            pred, best_vote = identify_query_hash(
                query_path=qpath,
                index=index,
                song_map=song_map,
                dt_min=dt_min,
                dt_max=dt_max,
                df_max=df_max,
                fan_out=fan_out,
            )
            dt = time.time() - t0
            times.append(dt)
            votes.append(best_vote)
            if pred == gt:
                hits += 1

        avg_t = float(np.mean(times)) if times else 0.0
        avg_vote = float(np.mean(votes)) if votes else 0.0

        all_rows.append(Row(
            distortion="Combined",
            config=cfg_name,
            dt_min=dt_min,
            dt_max=dt_max,
            df_max=df_max,
            fan_out=fan_out,
            hits=hits,
            total=total,
            accuracy=hits / total if total else 0.0,
            avg_query_time_s=avg_t,
            avg_best_vote=avg_vote,
        ))
        print(f"  => Combined: hits {hits}/{total}, acc={hits/total:.2f}, avg_time={avg_t:.2f}s, avg_vote={avg_vote:.1f}", flush=True)

    write_csv(os.path.join(OUT_DIR, "all_results.csv"), all_rows)

    per_distortion = defaultdict(list)
    for r in all_rows:
        per_distortion[r.distortion].append(r)

    for distortion, rows in per_distortion.items():
        fname = f"{distortion.lower()}_table.csv"
        write_csv(os.path.join(OUT_DIR, fname), rows)

    print("\nDone. CSVs written to:", OUT_DIR, flush=True)

task2() 

Original: 20 queries found
Noise: 20 queries found
Coding: 20 queries found
Mobile: 20 queries found

=== HASH CONFIG std (db_std.pkl) ===
Loaded index: 2184674 unique hashes, 574 songs
  => Original: hits 20/20, acc=1.00, avg_time=0.03s, avg_vote=112.8
  => Noise: hits 20/20, acc=1.00, avg_time=0.03s, avg_vote=55.4
  => Coding: hits 20/20, acc=1.00, avg_time=0.02s, avg_vote=47.4
  => Mobile: hits 13/20, acc=0.65, avg_time=0.01s, avg_vote=11.3
  => Combined: hits 73/80, acc=0.91, avg_time=0.04s, avg_vote=56.7

=== HASH CONFIG wide_t (db_wide_t.pkl) ===
Loaded index: 2897206 unique hashes, 574 songs
  => Original: hits 20/20, acc=1.00, avg_time=0.02s, avg_vote=127.5
  => Noise: hits 20/20, acc=1.00, avg_time=0.03s, avg_vote=63.6
  => Coding: hits 20/20, acc=1.00, avg_time=0.02s, avg_vote=54.4
  => Mobile: hits 14/20, acc=0.70, avg_time=0.01s, avg_vote=12.1
  => Combined: hits 74/80, acc=0.93, avg_time=0.02s, avg_vote=64.4

=== HASH CONFIG wide_f (db_wide_f.pkl) ===
Loaded index: 3007011

## Conclusion

The evaluation results from Task 2 provided the empirical evidence needed to finalize our strategy for the 3,319-track scale-up. Across all configurations, Clean, Noise, and Coding queries achieved a perfect 1.00 accuracy, demonstrating that the core hashing logic is extremely resilient to digital distortions. The true differentiator was the Mobile category, where accuracy scaled from 0.65 (std) to 0.75 (super_dense).

While super_dense offered the highest accuracy and the strongest "confidence" (indicated by an avg_vote of 34.0 for mobile queries), it came at a significant computational cost, with a lookup time of 0.19s. Nearly six times slower than the standard model. We observed that increasing the "Target Zone" (the wide_t and wide_f configs) consistently improved mobile performance to 0.70 without the massive memory bloat seen in super_dense.

These findings led us to design the smart_dense configuration for Task 3. By combining the increased time/frequency ranges of the "wide" configurations with a moderate increase in "fan-out" density (fanout = 8), we aimed to capture the 0.75 accuracy of the super-dense model while keeping the indexing time and RAM usage within our hrdware limits. Essentially, smart_dense was engineered to maximize the "votes" for noisy queries while maintaining the sub-0.05s search speeds observed in our more efficient test runs.


## Task 3: Scale-Up (30%)

In Task 2, we tested different hashing configurations on **30-second track snippets**.
The best-performing configuration, **`super_dense`**, uses a larger frequency window and increased fan_out.

The final **`super_dense`** configuration looks as follows: 
- dt_min: 10
- dt_max: 60
- df_max: 100
- fanout: 8


The pyhton script to generate the whole scaled up database is provided in the gen_final_database.py file in the ZIP.  

In this notebook, we scale this configuration to:

- Index **full-length tracks**.
- Include six tarballs from the MTG-Jamendo dataset.
- Collect statistics on database size, total hashes, and build time.
- Save a final index and CSV with results.

We will leverage **checkpoint files** to avoid recomputing the full database, since building the index from scratch is time-consuming.


### Task 3: Large-Scale Indexing Results (MTG-Jamendo Dataset)

The final scale-up utilized the **smart_dense** configuration on a high-performance desktop environment to index the full 3,319-track dataset. The results demonstrate the system's ability to maintain high fingerprint density while remaining within hardware constraints.

### Key Performance Metrics
| Metric | Value |
| :--- | :--- |
| **Total Tracks Indexed** | 3,319 |
| **Total Hash Entries** | 83,636,108 |
| **Unique Hash Keys** | 9,198,074 |
| **Avg. Hashes per Track** | ~25,199 |
| **Index Storage Size** | ~1,1 GB|
| **Total Build Time** | 18,195.11 s (~5.05 hours) |

### Results Analysis
* **Hash Density:** By achieving **~25,199 hashes per track**, the system maintains a redundancy level similar to the *super_dense* configuration from Task 2, which is critical for robust matching in noisy environments.
* **Hash Collisions:** While there are over 83 million total entries, they are mapped to only 9.2 million unique keys. This shows that many audio patterns are repeated across the dataset, justifying the use of an **Inverted Index** to group these occurrences efficiently.
* **Build Throughput:** The indexing rate averaged approximately **5.48 seconds per song**.